In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv


In [15]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [16]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [47]:
df = train

In [48]:
df['location'] = df.location.apply(lambda x: '' if x != x else x.lower())
df.loc[df.location == 'new York, ny','location'] = 'new york'
df.loc[df.location == 'nyc','location'] = 'new york'
df.loc[df.location == 'ny','location'] = 'new york'
df.loc[df.location == 'new york city','location'] = 'new york'
df.loc[df.location == 'us','location'] = 'united states'
df.loc[df.location == 'usa','location'] = 'united states'
df.loc[df.location == 'california','location'] = 'united states'
df.loc[df.location == 'southern california','location'] = 'united states'
df.loc[df.location == 'orange county','location'] = 'united states'
df.loc[df.location == 'u.s.a','location'] = 'united states'
df.loc[df.location == 'ca','location'] = 'united states'
df.loc[df.location == 'san francisco bay area','location'] = 'san francisco'
df.loc[df.location == 'uk','location'] = 'united kingdom'

In [49]:
df.loc[df.location == 'ss','location'] = ''
df.loc[df.location == 'everywhere','location'] = ''
df.loc[df.location == 'earth','location'] = ''
df.loc[df.location == '304','location'] = ''
df.loc[df.location == 'world','location'] = ''
df.loc[df.location == 'worldwide','location'] = ''
df.loc[df.location == '??????','location'] = ''
df.loc[df.location == 'pedophile hunting ground','location'] = ''
df.loc[df.location == 'planet earth','location'] = ''
df.loc[df.location == 'global','location'] = ''
df.loc[df.location == 'in the word of god','location'] = ''
df.loc[df.location == '?','location'] = ''
df.loc[df.location == 'mad as hell','location'] = ''
df.loc[df.location == '??','location'] = ''
df.loc[df.location == '?????','location'] = ''
df.loc[df.location == 'happily married with 2 kids','location'] = ''
df.loc[df.location == 'south','location'] = ''
df.loc[df.location == 'world wide','location'] = ''

In [50]:
df = df.replace(np.nan, '', regex=True)
df['keyword'] = df.keyword.str.replace('%20', '_')
df['keyword'] = df.keyword.str.lower()
df['text'] = df['keyword'] + ' ' + df['location'] + ' ' + df['text']

In [51]:
df.sample(10)

,id,keyword,location,text,target
1542,2229,chemical_emergency,"seattle, washington","chemical_emergency seattle, washington Downtow...",0
4853,6912,mass_murderer,,mass_murderer @atljw @cnnbrk fine line btw ma...,0
2713,3897,detonation,,detonation Ignition Knock (Detonation) Sensor...,0
1842,2648,crashed,international,crashed international TTW Today's News: Bin L...,1
4729,6725,lava,united states,lava united states Deal of The Day : http://t....,0
2982,4282,drowning,yorkshire\n,drowning yorkshire\n #Islamic #state issue a n...,0
5184,7400,obliterated,new york,obliterated new york Obliterated,0
6405,9156,suicide_bomber,"garden city, ny","suicide_bomber garden city, ny Suicide bomber ...",1
3836,5459,first_responders,"franklin, tn near nashville","first_responders franklin, tn near nashville A...",1
5130,7317,nuclear_reactor,"denver, co","nuclear_reactor denver, co Nuclear #Solar Powe...",1


In [52]:
df = df.drop(['id','keyword','location'], axis=1)

In [53]:
def clean_str(string):
    string = re.sub(r'https?\://\S+', '', string)
    string = re.sub(r'http?\://\S+', '', string)
    string = re.sub(r'@\w*\s', '', string)
    string = re.sub(r'#\w*\s', '', string)
    string = re.sub(r'\d', '', string)
    return string

df['text'] = df['text'].apply(lambda x: clean_str(str(x)))

In [54]:
df['text'].dropna(inplace=True)
df['text'] = [entry.lower() for entry in df['text']]
df['text']= [word_tokenize(entry) for entry in df['text']]
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df['text']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    df.loc[index,'text_final'] = str(Final_words)

In [109]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df['text_final'],df['target'],test_size=0.33)

In [110]:
Tfidf_vect = TfidfVectorizer(max_features=4000)
Tfidf_vect.fit(df['text_final'])
X_train_Tfidf = Tfidf_vect.transform(X_train)
X_test_Tfidf = Tfidf_vect.transform(X_test)

In [111]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train_Tfidf,y_train)

MultinomialNB()

In [112]:
print("Naive Bayes Accuracy Score on test : ",accuracy_score(Naive.predict(X_test_Tfidf), y_test))

Naive Bayes Accuracy Score on test :  0.8026263430163152


In [113]:
SVM = svm.SVC(C=1.0, kernel='sigmoid', degree=3, gamma='scale')
SVM.fit(X_train_Tfidf,y_train)

SVC(kernel='sigmoid')

In [114]:
print("SVM Accuracy Score on test : ",accuracy_score(SVM.predict(X_test_Tfidf), y_test))

SVM Accuracy Score on test :  0.7922801432550736


**Submit kaggle**

In [115]:
df = test

In [116]:
df['location'] = df.location.apply(lambda x: '' if x != x else x.lower())
df.loc[df.location == 'new York, ny','location'] = 'new york'
df.loc[df.location == 'nyc','location'] = 'new york'
df.loc[df.location == 'ny','location'] = 'new york'
df.loc[df.location == 'new york city','location'] = 'new york'
df.loc[df.location == 'us','location'] = 'united states'
df.loc[df.location == 'usa','location'] = 'united states'
df.loc[df.location == 'california','location'] = 'united states'
df.loc[df.location == 'southern california','location'] = 'united states'
df.loc[df.location == 'orange county','location'] = 'united states'
df.loc[df.location == 'u.s.a','location'] = 'united states'
df.loc[df.location == 'ca','location'] = 'united states'
df.loc[df.location == 'san francisco bay area','location'] = 'san francisco'
df.loc[df.location == 'uk','location'] = 'united kingdom'

In [117]:
df.loc[df.location == 'ss','location'] = ''
df.loc[df.location == 'everywhere','location'] = ''
df.loc[df.location == 'earth','location'] = ''
df.loc[df.location == '304','location'] = ''
df.loc[df.location == 'world','location'] = ''
df.loc[df.location == 'worldwide','location'] = ''
df.loc[df.location == '??????','location'] = np.nan
df.loc[df.location == 'pedophile hunting ground','location'] = ''
df.loc[df.location == 'planet earth','location'] = ''
df.loc[df.location == 'global','location'] = ''
df.loc[df.location == 'in the word of god','location'] = ''
df.loc[df.location == '?','location'] = ''
df.loc[df.location == 'mad as hell','location'] = ''
df.loc[df.location == '??','location'] = ''
df.loc[df.location == '?????','location'] = ''
df.loc[df.location == 'happily married with 2 kids','location'] = ''
df.loc[df.location == 'south','location'] = ''
df.loc[df.location == 'world wide','location'] = ''

In [118]:
df = df.replace(np.nan, '', regex=True)
df['keyword'] = df.keyword.str.replace('%20', '_')
df['keyword'] = df.keyword.str.lower()
df['text'] = df['keyword'] + ' ' + df['location'] + ' ' + df['text']

In [119]:
df = df.drop(['id','keyword','location'], axis=1)

In [120]:
def clean_str(string):
    string = re.sub(r'https?\://\S+', '', string)
    string = re.sub(r'http?\://\S+', '', string)
    string = re.sub(r'@\w*\s', '', string)
    string = re.sub(r'#\w*\s', '', string)
    string = re.sub(r'\d', '', string)
    return string

df['text'] = df['text'].apply(lambda x: clean_str(str(x)))

In [121]:
df['text'].dropna(inplace=True)
df['text'] = [entry.lower() for entry in df['text']]
df['text']= [word_tokenize(entry) for entry in df['text']]
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df['text']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    df.loc[index,'text_final'] = str(Final_words)

In [122]:
X_test_Tfidf = Tfidf_vect.transform(df['text_final'])

In [130]:
predictions_nb = Naive.predict(X_test_Tfidf)

In [131]:
predictions_nb = pd.DataFrame(predictions_nb, columns=['target'])

In [132]:
test['target'] = predictions_nb['target']

In [133]:
results = test[['id', 'target']]

In [129]:
results.to_csv('/kaggle/working/results_1.csv', index=False)